# discounts.ipynb
List books currently at minimum observed price.

In [1]:
from datetime import datetime

import holoviews as hv
import polars as pl

import monitorbookprices as mbp

We evaluate today's date with:

In [2]:
date = datetime.today().date()

Next, we load the tables `books` and `prices`, collecting the list of books and of recorded prices, respectively.

In [3]:
books = mbp.read_database(
    table_name='books',
    url='sqlite:///database.db',
    schema=mbp.schema(),
)
prices = mbp.read_database(
    table_name='prices',
    url='sqlite:///database.db',
    # schema=mbp.schema_prices(),
)

In [4]:
books

isbn,author,title,year,publisher,full_price,min_price,adelphi,buecher,feltrinelli,ibs,libraccio,mondadori,osiander
str,str,str,str,str,f64,f64,str,str,str,str,str,str,str
"""9783866473256""","""Karl Marx""","""Das Kapital""","""2009""","""Anaconda""",7.95,4.99,"""""","""https://www.buecher.de/artikel…",null,null,null,null,"""https://www.osiander.de/shop/h…"
"""9781857988826""","""Ursula K. Le Guin""","""The Dispossessed""","""1999""","""Orion Publishing Co""",13.0,9.49,null,"""https://www.buecher.de/artikel…","""https://www.lafeltrinelli.it/d…","""https://www.ibs.it/dispossesse…",null,null,"""https://www.osiander.de/shop/h…"


In [5]:
prices

isbn,site,price,date
str,str,f64,str
"""9783866473256""","""buecher""",7.95,"""2024-10-10"""
"""9783866473256""","""osiander""",4.99,"""2024-10-10"""
"""9781857988826""","""buecher""",13.0,"""2024-10-10"""
"""9781857988826""","""feltrinelli""",12.49,"""2024-10-10"""
"""9781857988826""","""ibs""",11.87,"""2024-10-10"""
…,…,…,…
"""9783866473256""","""osiander""",7.95,"""2024-10-19"""
"""9781857988826""","""buecher""",9.49,"""2024-10-19"""
"""9781857988826""","""feltrinelli""",12.49,"""2024-10-19"""


Let us take for example the book `Das Kapital` with ISBN `9783866473256`, with only recorded prices from `osiander.de`:

In [6]:
prices.filter(pl.col('isbn') == '9783866473256').filter(
    pl.col('site') == 'osiander'
)

isbn,site,price,date
str,str,f64,str
"""9783866473256""","""osiander""",4.99,"""2024-10-10"""
"""9783866473256""","""osiander""",4.99,"""2024-10-11"""
"""9783866473256""","""osiander""",4.99,"""2024-10-12"""
"""9783866473256""","""osiander""",4.99,"""2024-10-13"""
"""9783866473256""","""osiander""",4.99,"""2024-10-14"""
"""9783866473256""","""osiander""",4.99,"""2024-10-15"""
"""9783866473256""","""osiander""",4.99,"""2024-10-16"""
"""9783866473256""","""osiander""",7.95,"""2024-10-17"""
"""9783866473256""","""osiander""",7.95,"""2024-10-18"""


We can plot this using the function `plot_history` to see the plot of recent price recording in a database. We can specify the attribute `site` if want to see the records for one specific website, and the black dashed line will specify the `min_price`.

In [7]:
isbn_1 = '9783866473256'
book_1 = books.filter(pl.col('isbn') == isbn_1)
plot_1 = mbp.plot_history(
    prices.filter(pl.col('isbn') == isbn_1),
    book=book_1,
    hlines=['min'],
    width=1200,
    height=400,
)
plot_1

:Overlay
   .NdOverlay.I :NdOverlay   [site]
      :Curve   [date]   (price)
   .Curve.Min   :Curve   [x]   (y)

Oh no! We missed a big discount on Osiander! Now it seems to be back at full price.

How about the other book?

In [8]:
isbn_2 = '9781857988826'
book_2 = books.filter(pl.col('isbn') == isbn_2)
plot_2 = mbp.plot_history(
    prices.filter(pl.col('isbn') == isbn_2),
    book=book_2,
    hlines=['min'],
    width=1200,
    height=400,
)
plot_2

:Overlay
   .NdOverlay.I :NdOverlay   [site]
      :Curve   [date]   (price)
   .Curve.Min   :Curve   [x]   (y)

We have discounts on both osiander and buecher. Cool!

As a sidenote, we can stack all plots in a column in the following way:

In [9]:
hv.Layout([plot_1, plot_2]).cols(1)

:Layout
   .Overlay.I  :Overlay
      .NdOverlay.I :NdOverlay   [site]
         :Curve   [date]   (price)
      .Curve.Min   :Curve   [x]   (y)
   .Overlay.II :Overlay
      .NdOverlay.I :NdOverlay   [site]
         :Curve   [date]   (price)
      .Curve.Min   :Curve   [x]   (y)